# calculate rotation and transposition matrix

In [8]:
import os

data_folder = r'D:\Pu\20211208-P_brain_CTP11-500_M1_DNA_RNA-seq_hybrid'
before_position_file = os.path.join(data_folder, 'positions_before_align.txt')
after_position_file = os.path.join(data_folder, 'positions_after_align.txt')


In [9]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [10]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


In [11]:
R, T

(array([[ 0.99986777, -0.01626156],
        [ 0.01626156,  0.99986777]]),
 array([-389.64807599,   53.53981124]))

# transpose 60x positions

In [12]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [13]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-7512.06312574 -4194.10024285]
 [-7312.08957133 -4197.3525555 ]
 [-7112.11601692 -4200.60486815]
 [-6912.14246251 -4203.85718079]
 [-6712.16890811 -4207.10949344]
 [-7508.81081309 -3994.12668845]
 [-7308.83725868 -3997.37900109]
 [-7108.86370428 -4000.63131374]
 [-6908.89014987 -4003.88362639]
 [-6708.91659546 -4007.13593903]
 [-6508.94304105 -4010.38825168]
 [-7705.53205485 -3790.90082139]
 [-7505.55850045 -3794.15313404]
 [-7305.58494604 -3797.40544669]
 [-7105.61139163 -3800.65775933]
 [-6905.63783722 -3803.91007198]
 [-6705.66428281 -3807.16238462]
 [-6505.69072841 -3810.41469727]
 [-6305.717174   -3813.66700992]
 [-7702.27974221 -3590.92726699]
 [-7502.3061878  -3594.17957963]
 [-7302.33263339 -3597.43189228]
 [-7102.35907898 -3600.68420492]
 [-6902.38552458 -3603.93651757]
 [-6702.41197017 -3607.18883022]
 [-6502.43841576 -3610.44114286]
 [-6302.46486135 -3613.69345551]
 [-6102.49130694 -3616.94576815]
 [-7899.00098397 -3387.70139993]
 [-7699.02742956 -3390.95371258]
 [-7499.05

In [14]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20211208-P_brain_CTP11-500_M1_DNA_RNA-seq_hybrid\translated_positions_all.txt


# further adjust manually

In [8]:
manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [9]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

\\storm6-pc\STORM6-FLASHDrive\Pu\20201127-NOAcr_CTP-08_E14_brain_no_clearing\adjusted_translated_positions_all.txt
